读取CIFAR-10数据

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

transform = transforms.Compose([
    transforms.Resize(224), #与ImageNet数据集的尺寸一致
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

batch_size = 64

trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:53<00:00, 3.17MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


使用预训练的ResNet

In [7]:
# 加载预训练的ResNet模型
model = models.resnet18(pretrained=True)

# 冻结所有卷积层的参数(减少计算量)
for param in model.parameters():
    param.requires_grad = False

# 替换最后一层全连接层以适应新的分类任务
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(trainset.classes))

# 将模型移动到GPU（如果可用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

训练并测试

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

model.to(device)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, model, criterion, optimizer)
    test_loop(testloader, model, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 0.703250  [   64/50000]
loss: 0.645929  [ 6464/50000]
loss: 0.639983  [12864/50000]
loss: 0.320120  [19264/50000]
loss: 0.549756  [25664/50000]
loss: 0.350397  [32064/50000]
loss: 0.387203  [38464/50000]
loss: 0.355298  [44864/50000]
Test Error: 
 Accuracy: 80.7%, Avg loss: 0.568583 

Epoch 2
-------------------------------
loss: 0.421394  [   64/50000]
loss: 0.792328  [ 6464/50000]
loss: 0.658402  [12864/50000]
loss: 0.579646  [19264/50000]
loss: 0.572500  [25664/50000]
loss: 0.459337  [32064/50000]
loss: 0.404759  [38464/50000]
loss: 0.536964  [44864/50000]
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.562279 

Epoch 3
-------------------------------
loss: 0.683687  [   64/50000]
loss: 0.598157  [ 6464/50000]
loss: 0.458683  [12864/50000]
loss: 0.407793  [19264/50000]
loss: 0.650905  [25664/50000]
loss: 0.515264  [32064/50000]
loss: 0.521758  [38464/50000]
loss: 0.311819  [44864/50000]
Test Error: 
 Accuracy: 80.8%, Avg loss: 0.562382 

Epoc